In [1]:
import pandas as pd
import numpy as np
from pyvis.network import Network
from pybliometrics.scopus import AuthorSearch
from pybliometrics.scopus import AuthorRetrieval
from pprint import pprint
import gensim
from gensim import corpora
import itertools
import time
from sklearn.cluster import KMeans
import sys
sys.path.append("..")
from rcn_py import scopus

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Find the author_id by the name 

In [3]:

s = AuthorSearch('AUTHLAST(Kalverla) and AUTHFIRST(Peter)')  #todo also check Joris van Eijnatten

In [5]:
print(s)

Search 'AUTHLAST(Kalverla) and AUTHFIRST(Peter)' yielded 1 author as of 2023-02-28:
    Kalverla, Peter; AUTHOR_ID:57189467443 (8 document(s))


In [6]:
AuthorRetrieval(7004691405)

In [54]:
author_id = 57189467443
au = AuthorRetrieval(author_id)
print(au.self_link)


https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57189467443&origin=inward


In [7]:
node_retrieved = []
def all_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Access to documents for the last five years
    new_docs = docs[(docs.coverDate > '2018')]
    au_id = new_docs.author_ids
    
    link = []
    all_node = []

    node_retrieved.append(author_id)
    
    for i in au_id:
        coau_id = i.split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if AuthorRetrieval(j).affiliation_current:
                # Geo-filtering 
                # todo: all the affiliations 
                # if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                    if j not in all_node:
                        all_node.append(j)
                    new_coau_id.append(j)
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1)
    return all_node, link

In [8]:
node, link = all_coauthor(author_id, 0, node_retrieved)

In [42]:
def nld_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Access to documents for the last five years
#     new_docs = docs[(docs.coverDate > '2018')]
    au_id = scopus.author_ids
    
    link = []
    all_node = []

    node_retrieved.append(author_id)
    
    for i in au_id:
        coau_id = i.split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if AuthorRetrieval(j).affiliation_current:
                # Geo-filtering 
                # todo: all the affiliations 
                if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                    if j not in all_node:
                        all_node.append(j)
                    new_coau_id.append(j)
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1)
    return all_node, link

In [21]:
node, link = nld_coauthor(author_id, 0, node_retrieved)

In [10]:
for i in node:
    print(AuthorRetrieval(i).given_name+' '+AuthorRetrieval(i).surname)

Rolf W. Hut
Niels Drost
Nick C. Van De Giesen
Ben J.C. van Werkhoven
Banafsheh Abdollahi
Jerom P.M. Aerts
Thomas Albers
Fakhereh Alidoost
Bouwe Andela
Jaro Camphuijsen
Yifat Dzigan
Ronald van Haren
Eric W.H. Hutton
Peter Kalverla
Maarten A.J. van Meersbergen
Gijs van den Oord
Inti Pelupessy
Stef Smeets
Stefan Verhoeven
Martine G. de Vos
Berend Weel
Mark Schelbergen
Roland Schmehl
Simon Watson
Albert A.M. Holtslag
Reinder J. Ronda
G. J. Steeneveld
James B. Duncan


In [12]:
def get_hindex(au_id):
    au = AuthorRetrieval(au_id)
    return au.h_index

In [22]:
sources = []
targets = []
weights = []

for i in link:
    sources.append(i[0])
    targets.append(i[1])
    weights.append(link.count(i))

In [25]:
# Pyvis network
N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True)
N.toggle_hide_edges_on_drag(False)
N.barnes_hut()

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    N.add_node(src, src, title=src, group = au_group[src])
    N.add_node(dst, dst, title=dst, group = au_group[dst])
    N.add_edge(src, dst, value=w)

neighbor_map = N.get_adj_list()

# add neighbor data to node hover data
for node in N.nodes:
    neighbors = []
    for neighbor_id in neighbor_map[node["id"]]:
        neigh = AuthorRetrieval(neighbor_id)
        neighbors.append(neigh.given_name+' '+neigh.surname)
    #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
    node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
    node["value"] = get_hindex(node["id"])
    node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
    # node["group"] = group[node["id"]]
    # node["font_size"] = get_hindex(node["id"])

N.show("peter.html")

In [14]:
au = AuthorRetrieval(author_id)
docs = pd.DataFrame(au.get_documents())

In [15]:
s.authors[0].eid.split('-')[-1]

'57189467443'

In [11]:
def filter_country(author_list, country_code):
    filtered_authors = []
    for i in author_list:
        aff_list = AuthorRetrieval(i).affiliation_current
        for aff_cur in aff_list:
            if aff_cur.country_code == country_code:
                filtered_authors.append(i)
                break
    return filtered_authors
        

In [22]:
node

[23488924100,
 15065185400,
 6603507671,
 55052390500,
 57201991588,
 57220200254,
 57807312200,
 57189690259,
 57215841810,
 57807781300,
 57220157018,
 55227059300,
 57189467443,
 55065848900,
 57195348393,
 57211407731,
 57807466500,
 23096348600,
 57533863700,
 55171306200,
 56046806200,
 7003773804,
 8312994100,
 7006904374,
 6602265575,
 8879755400,
 57640658500]

In [37]:
link

[(6603507671, 15065185400),
 (6603507671, 23488924100),
 (6603507671, 55052390500),
 (6603507671, 57201991588),
 (15065185400, 23488924100),
 (15065185400, 55052390500),
 (15065185400, 57201991588),
 (23488924100, 55052390500),
 (23488924100, 57201991588),
 (55052390500, 57201991588),
 (7003773804, 8312994100),
 (7003773804, 56046806200),
 (7003773804, 57189467443),
 (8312994100, 56046806200),
 (8312994100, 57189467443),
 (56046806200, 57189467443),
 (6602265575, 7006904374),
 (6602265575, 8879755400),
 (6602265575, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (8879755400, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (7006904374, 57640658500),
 (8879755400, 57189467443),
 (8879755400, 57640658500),
 (57189467443, 57640658500),
 (6602265575, 7006904374),
 (6602265575, 8879755400),
 (6602265575, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (8879755400, 57189467443)]

In [23]:
author_lst = filter_country(node, 'nld')
print(len(author_lst))

27


In [24]:
def get_coauthor(author_first, author_last, depth):
    s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
    author_id = s.authors[0].eid.split('-')[-1]
    node2 = []
    node, link = nld_coauthor(author_id, depth, node2)
    sources = []
    targets = []
    weights = []

    for i in link:
        sources.append(i[0])
        targets.append(i[1])
        weights.append(link.count(i))
    
    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True)
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        N.add_node(src, src, title=src)
        N.add_node(dst, dst, title=dst)
        N.add_edge(src, dst, value=w)

    neighbor_map = N.get_adj_list()

    # add neighbor data to node hover data
    for node in N.nodes:
        neighbors = []
        for neighbor_id in neighbor_map[node["id"]]:
            neigh = AuthorRetrieval(neighbor_id)
            neighbors.append(neigh.given_name+' '+neigh.surname)
        #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
        node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
        node["value"] = get_hindex(node["id"])
        node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
       

    N.show(author_last + ".html")


In [25]:
get_coauthor("Bart","Schilperoort",0) 

In [26]:
docs.description

0    Hutton et al. (2016) argued that computational...
1    Airborne wind energy (AWE) systems harness ene...
2    Offshore wind energy production is rapidly gro...
3    Ten years of ERA5 reanalysis data are combined...
4    Numerical weather prediction models play an im...
5    Uncertainty reduction in offshore wind systems...
6    In numerical models of geophysical fluid syste...
7    In the winter of 2012/13, the Katabatic Winds ...
Name: description, dtype: object

In [ ]:
# Import package/library
import network_analysis as na

author = na.find_author_by_name(firstname="Joris", lastname="Eijnatten")
author = na.find_author_by_orcid(orcid = ....)
coauthors = na.find_coauthors(author)
coauthors = na.filter_coauthors(coauthors, country_code="nld")
graph = na.visualize(coauthors)

## Topic Modeling

In [30]:
docs.description[0]

'Hutton et al. (2016) argued that computational hydrology can only be a proper science if the hydrological community makes sure that hydrological model studies are executed and presented in a reproducible manner. Hut, Drost and van de Giesen replied that to achieve this hydrologists should not "re-invent the water wheel" but rather use existing technology from other fields (such as containers and ESMValTool) and open interfaces (such as the Basic Model Interface, BMI) to do their computational science (Hut et al., 2017). With this paper and the associated release of the eWaterCycle platform and software package (available on Zenodo: https://doi.org/10.5281/zenodo.5119389, Verhoeven et al., 2022), we are putting our money where our mouth is and providing the hydrological community with a "FAIR by design" (FAIR meaning findable, accessible, interoperable, and reproducible) platform to do science. The eWaterCycle platform separates the experiments done on the model from the model code. In

In [31]:
docs.title[0]

'The eWaterCycle platform for open and FAIR hydrological collaboration'

In [9]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

In [10]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def clean_text(text):
    text = text.replace('\n'," ") 
    text = re.sub(r"-", " ", text) 
    text = re.sub(r"\d+/\d+/\d+", "", text) 
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) 
    text = re.sub(r"[\w]+@[\.\w]+", "", text) 
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)
    pure_text = ''
    for letter in text:
        # Leave only letters and spaces
        if letter.isalpha() or letter==' ':
            letter = letter.lower()
            pure_text += letter
            
    corpus_lst = [ps.stem(word) for word in pure_text.split() if word not in stop_words]
    return corpus_lst

In [30]:
cleaned_abstract = clean_text(docs.description[0])
print(cleaned_abstract)

['hutton', 'et', 'al', 'argu', 'comput', 'hydrolog', 'proper', 'scienc', 'hydrolog', 'commun', 'make', 'sure', 'hydrolog', 'model', 'studi', 'execut', 'present', 'reproduc', 'manner', 'hut', 'drost', 'van', 'de', 'giesen', 'repli', 'achiev', 'hydrologist', 'invent', 'water', 'wheel', 'rather', 'use', 'exist', 'technolog', 'field', 'contain', 'esmvaltool', 'open', 'interfac', 'basic', 'model', 'interfac', 'bmi', 'comput', 'scienc', 'hut', 'et', 'al', 'paper', 'associ', 'releas', 'ewatercycl', 'platform', 'softwar', 'packag', 'avail', 'zenodo', 'httpsdoiorgzenodo', 'verhoeven', 'et', 'al', 'put', 'money', 'mouth', 'provid', 'hydrolog', 'commun', 'fair', 'design', 'fair', 'mean', 'findabl', 'access', 'interoper', 'reproduc', 'platform', 'scienc', 'ewatercycl', 'platform', 'separ', 'experi', 'done', 'model', 'model', 'code', 'ewatercycl', 'hydrolog', 'model', 'access', 'common', 'interfac', 'bmi', 'python', 'run', 'insid', 'softwar', 'contain', 'way', 'model', 'access', 'similar', 'manner'

In [31]:
cleaned_abs_corpus = []
for abstract_lst in docs.description:
    cleaned_abs_corpus.append(clean_text(abstract_lst))
    
num_topics = 4
dictionary = corpora.Dictionary(cleaned_abs_corpus) 
corpus = [dictionary.doc2bow(text) for text in cleaned_abs_corpus]
    
t0= time.time()
passes= 150
np.random.seed(1)

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=num_topics,
                                       chunksize= 4000, 
                                       batch= True,
                                       minimum_probability=0.001,
                                       iterations=350,
                                       passes=passes)
t1 = time.time()
# Print the Keyword in the 4 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print("time for ", passes, " passes: ", (t1-t0), "seconds.")

[(0,
  '0.058*"wind" + 0.018*"speed" + 0.014*"jet" + 0.013*"era" + 0.013*"observ" + '
  '0.013*"model" + 0.011*"low" + 0.011*"use" + 0.011*"level" + 0.011*"energi"'),
 (1,
  '0.035*"wind" + 0.012*"condit" + 0.010*"surfac" + 0.010*"found" + '
  '0.010*"parameter" + 0.010*"pbl" + 0.010*"model" + 0.008*"character" + '
  '0.008*"extrem" + 0.008*"scheme"'),
 (2,
  '0.002*"high" + 0.002*"forecast" + 0.002*"simul" + 0.002*"valu" + '
  '0.002*"repres" + 0.002*"perform" + 0.002*"lead" + 0.002*"method" + '
  '0.002*"system" + 0.002*"rang"'),
 (3,
  '0.042*"model" + 0.023*"ewatercycl" + 0.015*"platform" + 0.013*"process" + '
  '0.013*"hydrolog" + 0.011*"scienc" + 0.011*"parametr" + 0.009*"system" + '
  '0.009*"provid" + 0.009*"access"')]
time for  150  passes:  2.8978772163391113 seconds.


In [32]:
lda_model.show_topics(num_words=25, formatted=False)


[(0,
  [('wind', 0.05798044),
   ('speed', 0.018184189),
   ('jet', 0.014066984),
   ('era', 0.012694241),
   ('observ', 0.012693972),
   ('model', 0.012689519),
   ('low', 0.011322881),
   ('use', 0.011322086),
   ('level', 0.011322073),
   ('energi', 0.0113214245),
   ('profil', 0.009949115),
   ('repres', 0.008577664),
   ('data', 0.008577502),
   ('studi', 0.007204741),
   ('represent', 0.007204579),
   ('shear', 0.0072045075),
   ('height', 0.0072044916),
   ('north', 0.0072044916),
   ('sea', 0.0072044916),
   ('shape', 0.0072044916),
   ('cluster', 0.007204491),
   ('mast', 0.00720449),
   ('met', 0.00720449),
   ('perform', 0.0058327173),
   ('demonstr', 0.005832663)]),
 (1,
  [('wind', 0.0354852),
   ('condit', 0.012218964),
   ('surfac', 0.009891995),
   ('found', 0.009891904),
   ('pbl', 0.009889407),
   ('parameter', 0.009889407),
   ('model', 0.009886225),
   ('character', 0.007564224),
   ('extrem', 0.0075636273),
   ('scheme', 0.007563246),
   ('low', 0.007563111),
   ('

In [33]:
for i in range(len(cleaned_abs_corpus)):
    for j1,j2 in lda_model[corpus[i]]:
        print(j2)

0.001084416
0.0010798272
0.0010753459
0.9967604
0.99461305
0.0018029399
0.001778131
0.0018058829
0.993448
0.0022223713
0.0021596113
0.002170031
0.99625033
0.0012538695
0.0012352354
0.0012605928
0.99412096
0.0020128905
0.0019144075
0.0019518129
0.002094716
0.9939112
0.0019884403
0.0020056863
0.0023199155
0.0022932314
0.0022576414
0.9931292
0.0015599736
0.995364
0.0015277533
0.0015482353


In [34]:
group = []
for i in range(len(cleaned_abs_corpus)):
    scores = []
    for j1,j2 in lda_model[corpus[i]]:
        scores.append(j2)
    group.append(scores.index(max(scores)))
docs['group'] = group
docs

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor,group
0,2-s2.0-85134310662,10.5194/gmd-15-5371-2022,None,None,The eWaterCycle platform for open and FAIR hyd...,ar,Article,Hut R.,60159380;60118019;60000221,Netherlands eScience Center;Faculteit Civiele ...,...,Hutton et al. (2016) argued that computational...,None,3,1,repositoryvor,Green,None,027.017.F01,Netherlands eScience Center,3
1,2-s2.0-85093942697,10.5194/wes-5-1097-2020,None,None,Clustering wind profile shapes to estimate air...,ar,Article,Schelbergen M.,60117954;60004156,"Faculteit Luchtvaart- en Ruimtevaarttechniek, ...",...,Airborne wind energy (AWE) systems harness ene...,None,4,1,repositoryvor,Green,H2020,H2020-ITN-642682,Horizon 2020 Framework Programme,0
2,2-s2.0-85079363786,10.1002/qj.3748,None,None,Quality of wind characteristics in recent wind...,ar,Article,Kalverla P.C.,60004156,Wageningen University &amp; Research,...,Offshore wind energy production is rapidly gro...,DOWA | ERA-5 | low-level jets | Meteo Mast IJm...,22,1,publisherhybridgold,Hybrid Gold,NWO,SH‐312‐15,Nederlandse Organisatie voor Wetenschappelijk ...,0
3,2-s2.0-85070920049,10.5194/wes-4-193-2019,None,None,Low-level jets over the north sea based on ERA...,ar,Article,Kalverla P.C.,60022168;60004156,Energy research Centre of the Netherlands - EC...,...,Ten years of ERA5 reanalysis data are combined...,None,37,1,repositoryam,Green,NWO,SH-312-15,Nederlandse Organisatie voor Wetenschappelijk ...,0
4,2-s2.0-85052934619,10.1002/we.2267,None,None,Evaluation of three mainstream numerical weath...,ar,Article,Kalverla P.,60004156;60000203,Wageningen University &amp; Research;Royal Net...,...,Numerical weather prediction models play an im...,Met mast IJmuiden | Model evaluation | North S...,14,1,publisherhybridgold,Hybrid Gold,EZ,SH-312-15,Ministerie van Economische Zaken,0
5,2-s2.0-85015913347,10.1016/j.jweia.2017.03.008,S0167610516307061,None,An observational climatology of anomalous wind...,ar,Article,Kalverla P.C.,60004156;60000203,Wageningen University &amp; Research;Royal Net...,...,Uncertainty reduction in offshore wind systems...,None,29,1,publisherhybridgold,Hybrid Gold,EZ,14185,Ministerie van Economische Zaken,1
6,2-s2.0-84971254242,10.1002/qj.2817,None,None,A maximum entropy approach to the parametrizat...,ar,Article,Verkley W.,60000203,Royal Netherlands Meteorological Institute,...,In numerical models of geophysical fluid syste...,atmospheric modelling | maximum entropy | para...,12,0,None,None,None,undefined,None,3
7,2-s2.0-84975217639,10.1175/JAMC-D-15-0258.1,None,None,Evaluation of the weather research and forecas...,ar,Article,Kalverla P.C.,60102124;60004156;60003729,Université Fédérale Toulouse Midi-Pyrénées;Wag...,...,"In the winter of 2012/13, the Katabatic Winds ...",Atmosphere-land interaction | Boundary layer |...,25,1,publisherfree2read,Bronze,None,undefined,None,1


In [7]:
def lda_cluster(docs):
    cleaned_abs_corpus = []
    for i in range(len(docs)):
        if docs.description[i]:
            cleaned_abs_corpus.append(clean_text(docs.description[i]))
        else:
            cleaned_abs_corpus.append(clean_text(docs.title[i]))

    num_topics = 4
    dictionary = corpora.Dictionary(cleaned_abs_corpus) 
    corpus = [dictionary.doc2bow(text) for text in cleaned_abs_corpus]

    t0= time.time()
    passes= 150
    np.random.seed(1)

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics,
                                           chunksize= 4000, 
                                           batch= True,
                                           minimum_probability=0.001,
                                           iterations=350,
                                           passes=passes)
    group = []
    for i in range(len(cleaned_abs_corpus)):
        scores = []
        for j1,j2 in lda_model[corpus[i]]:
            scores.append(j2)
        group.append(scores.index(max(scores)))
    docs['group'] = group
    return docs

In [36]:
docs.group[0]

3

In [4]:
def nld_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Get clusters of docs
    docs = lda_cluster(docs)
    # Access to documents for the last five years
#     new_docs = docs[(docs.coverDate > '2018')]
    au_id = docs.author_ids
    
    link = []
    all_node = []
    groups = []
    au_group = {}

    node_retrieved.append(author_id)
    
    for i in range(len(au_id)):
        coau_id = au_id[i].split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if aff:
                # Geo-filtering 
                # todo: all the affiliations 
                # if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':   
                    if j not in all_node:
                        all_node.append(j)
                    if docs.group[i] not in groups:
                        groups.append(docs.group[i])
                    new_coau_id.append(j)
                    au_group[j] = groups.index(docs.group[i])
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1, node_retrieved)
    return all_node, link, au_group

In [16]:
node_retrieved = []
node, link, au_group = nld_coauthor(57189467443, 1, 5, node_retrieved)

In [26]:
link

[(6603507671, 15065185400),
 (6603507671, 23488924100),
 (6603507671, 55052390500),
 (6603507671, 57201991588),
 (15065185400, 23488924100),
 (15065185400, 55052390500),
 (15065185400, 57201991588),
 (23488924100, 55052390500),
 (23488924100, 57201991588),
 (55052390500, 57201991588),
 (7003773804, 8312994100),
 (7003773804, 56046806200),
 (7003773804, 57189467443),
 (8312994100, 56046806200),
 (8312994100, 57189467443),
 (56046806200, 57189467443),
 (6602265575, 7006904374),
 (6602265575, 8879755400),
 (6602265575, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (8879755400, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (7006904374, 57640658500),
 (8879755400, 57189467443),
 (8879755400, 57640658500),
 (57189467443, 57640658500),
 (6602265575, 7006904374),
 (6602265575, 8879755400),
 (6602265575, 57189467443),
 (7006904374, 8879755400),
 (7006904374, 57189467443),
 (8879755400, 57189467443)]

If the quota is exceeded, X-RateLimit-Limit and X-RateLimit-Remaining don’t return. 

In [58]:
print(s)

Koller M. from Department of Clinical Research in Switzerland,
published 209 document(s) since 2001 
which were cited by 4,580 author(s) in 4,995 document(s) as of 2023-02-26


In [5]:
def get_coauthor(author_first, author_last, depth):
    s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
    author_id = s.authors[0].eid.split('-')[-1]
    
    node_retrieved = []
    node, link, au_group = nld_coauthor(author_id, depth, node_retrieved)
    sources = []
    targets = []
    weights = []

    for i in link:
        sources.append(i[0])
        targets.append(i[1])
        weights.append(link.count(i))
    
    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white")
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        N.add_node(src, src, title=src, group = au_group[src])
        N.add_node(dst, dst, title=dst, group = au_group[dst])
        N.add_edge(src, dst, value=w)

    neighbor_map = N.get_adj_list()

    # add neighbor data to node hover data
    for node in N.nodes:
        neighbors = []
        for neighbor_id in neighbor_map[node["id"]]:
            neigh = AuthorRetrieval(neighbor_id)
            neighbors.append(neigh.given_name+' '+neigh.surname)
        #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
        node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
        node["value"] = get_hindex(node["id"])
        node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
       

    N.show(author_last + ".html")



Example

In [2]:
scopus.get_coauthor("Peter", "Kalverla", 0)

PermissionError: [Errno 13] Permission denied: '/Users/jennifer/.pybliometrics/Scopus/author_retrieval/ENHANCED/6603615408'

In [9]:

AuthorRetrieval(6603615408)


PermissionError: [Errno 13] Permission denied: '/Users/jennifer/.pybliometrics/Scopus/author_retrieval/ENHANCED/6603615408'